# Star Resonance Tool - Colab Launcher

このノートブックは、Google Colab上で「Star Resonance Tool」を実行するためのものです。
以下のセルを順番に実行してください。

### 手順
1. 再生ボタン(▶)を押してセルを実行します。
2. 処理が完了すると、最後に **`your url is: https://....trycloudflare.com`** というURLが表示されます。
3. そのリンクをクリックしてください。
4. アプリが起動します。

In [ ]:
# リポジトリのクローン
!git clone https://github.com/takeshich/star-resonance-tool.git
%cd star-resonance-tool

# 依存関係のインストール
!pip install -r requirements.txt

# cloudflared（トンネルツール）のダウンロード
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

In [ ]:
import subprocess
import time

# Streamlitをバックグラウンドで実行（CORS/XSRF保護を無効化）
subprocess.Popen(['streamlit', 'run', 'app.py', '--server.enableCORS=false', '--server.enableXsrfProtection=false', '--server.port=8501'])

# Cloudflare Tunnelを開始
print("Starting Cloudflare Tunnel...")
process = subprocess.Popen(['./cloudflared-linux-amd64', 'tunnel', '--url', 'http://localhost:8501'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# URLが表示されるまでログを監視
print("Waiting for Cloudflare Tunnel URL...")
limit = 0
found = False
for line in iter(process.stderr.readline, b''):
    line_str = line.decode('utf-8')
    if 'trycloudflare.com' in line_str:
        print("\n" + "="*40)
        print(f"Target URL: {line_str.strip().split()[-1]}")
        print("="*40 + "\n")
        found = True
        break
    limit += 1
    if limit > 200:
        break

if not found:
    print("URL not found yet. Please wait check the output above or below.")

# プロセスを維持
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped")